In [377]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列表示を省略しない
#pl.Config.set_tbl_rows(-1)# 行の表示を省略しない
import polars.selectors as cs#列選択のプリセット

# ２章　小売店のデータでデータ加工を行う１０本ノック
本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう
まずデータを読み込んで、データの概観を確認します。

In [378]:
uriage_data = pl.read_csv("uriage.csv")
uriage_data.head()

purchase_date,item_name,item_price,customer_name
str,str,i64,str
"""2019-06-13 18:…","""商品A""",100,"""深井菜々美"""
"""2019-07-13 13:…","""商 品 S""",null,"""浅田賢二"""
"""2019-05-11 19:…","""商 品 a""",null,"""南部慶二"""
"""2019-02-12 23:…","""商品Z""",2600,"""麻生莉緒"""
"""2019-04-22 03:…","""商品a""",null,"""平田鉄二"""


売上履歴のデータで、購入日、商品名、商品価格、顧客名の4列あります。\
先頭の数行を表示した時点で、商品Aと商品 aと商品aの表記揺れが見えます。\
さらに商品価格が記入されていない行があります。

In [379]:
kokyaku_data = pl.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

顧客名,かな,地域,メールアドレス,登録日
str,str,str,str,str
"""須賀ひとみ""","""すが ひとみ""","""H市""","""suga_hitomi@ex…","""2018/01/04"""
"""岡田 敏也""","""おかだ としや""","""E市""","""okada_toshiya@…","""42782"""
"""芳賀 希""","""はが のぞみ""","""A市""","""haga_nozomi@ex…","""2018/01/07"""
"""荻野 愛""","""おぎの あい""","""F市""","""ogino_ai@examp…","""42872"""
"""栗田 憲一""","""くりた けんいち""","""E市""","""kurita_kenichi…","""43127"""


顧客情報のデータで、顧客名、かな、地域、メールアドレス、登録日の5列あります。\
こちらも登録日が日付とシリアル値(日付)の揺れがあります。
また、顧客名の空白が全角、半角、空白無しでばらついています。

### ノック１２：データの揺れを見てみよう

先ほどのデータの概観で見つけた商品名をもう一度表示して、表記ゆれを見てみます。

In [380]:
uriage_data["item_name"].head(n = 5)

item_name
str
"""商品A"""
"""商 品 S"""
"""商 品 a"""
"""商品Z"""
"""商品a"""


アルファベットが小文字だったり、間に空白があったりの表記ゆれを見つけました。\

同様にして、商品価格の概観も見てみましょう。

In [381]:
uriage_data["item_price"].head(n = 5)

item_price
i64
100
null
null
2600
null


商品価格に欠損値が含まれています。\
このまま合計を計算すると売り上げを過小評価してしまうので、欠損値を埋める必要があります。

### ノック１３：データに揺れがあるまま集計しよう

データの揺れがどれくらい集計に影響を与えるかを確認してみます。\
表記ゆれを直さないまま、購入日をDateTime型に変換して、購入年月列を作成します。

In [382]:
uriage_data = (
    uriage_data
    # 文字列型の購入日をDateTime型に変換する
    .with_columns(
        pl.col("purchase_date")
        .str.to_datetime()
        .alias("purchase_date")
    )
    # 購入年月列を追加する
    .with_columns(
        pl.col("purchase_date")
        .dt.strftime("%Y%m")
        .alias("purchase_month")
    )
)

# 月毎の各商品の売り上げを集計する
res = (
    uriage_data
    .pivot(
        index = "purchase_month",
        columns = "item_name",
        values = "item_name",
        aggregate_function = "count",
        sort_columns = True
    )
    .sort("purchase_month")
)
res

purchase_month,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,商 品T,商 品V,商 品 S,商 品 a,商 品 q,商 品 s,商 品A,商 品C,商 品D,商 品E,商 品F,商 品G,商 品H,商 品I,商 品K,商 品M,商 品N,商 品O,商 品P,商 品T,商 品U,商 品V,商 品X,商 品Y,商 品s,商品 A,商品 B,商品 E,商品 F,商品 H,商品 I,商品 J,商品 K,商品 M,商品 O,商品 Q,商品 R,商品 S,商品 T,商品 V,商品 X,商品 a,商品 g,商品 o,商品 v,商品 w,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,商品K,商品L,商品M,商品N,商品O,商品P,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z,商品a,商品c,商品d,商品e,商品g,商品i,商品j,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""201901""",null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,1,1,1,null,null,2,null,1,null,null,null,null,null,1,null,null,1,1,null,null,1,null,1,null,null,null,null,null,null,18,13,18,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,null,null,1,null,null,null,null,null,1,1,1,null,null,null,null,null,null,null
"""201902""",null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,1,null,null,1,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,1,null,null,null,1,null,1,1,null,null,19,13,26,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,null,null,null,null,null,null,1,null,null,null,null,null,1,1,1,null,null
"""201903""",null,null,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,1,null,17,21,20,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""201904""",1,null,null,null,null,null,null,null,null,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,3,null,null,null,null,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16,19,24,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,null,1,null,null,1,1,null,null,null,null,null,null,null,1,null,null,null,null
"""201905""",null,null,null,null,null,1,null,null,null,null,null,null,null,2,null,1,1,null,null,null,null,null,null,null,1,1,null,null,null,null,null,1,null,null,null,null,null,null,1,null,null,1,1,null,null,1,null,null,1,null,null,1,null,null,null,1,20,14,15,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,null,null,1,null,null,null,null,null,null,1,null,null,null,null,null,null,null,1
"""201906""",null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,1,null,null,null,1,null,null,null,null,null,1,null,null,1,null,null,null,null,1,null,null,null,1,null,null,null,null,null,null,null,null,null,1,null,null,1,null,null,null,23,12,10,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,null,null,null,null,null,null,1,null,null,null,null,1,null,null,null,null,1,null
"""201907""",null,null,null,null,null,null,null,null,1,null,null,null,1,null,1,null,null,null,null,1,null,null,1,null,null,null,null,1,null,null,null,1,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,1,1,null,null,null,null,null,null,20,20,17,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,null,null,null,null,null,null,1,null,null,null,1,null,2,null,null,null,null,null


集計結果は100列あります(内1列は購入月で、商品名は99列あります)。\
商品名の列は、商品 + アルファベット(大文字) で26列になるはずですが、何倍にも増えています。\
これでは正しい結果を得られないので、次のノックから表記ゆれを修正していきましょう。

### ノック１４：商品名の揺れを補正しよう

表記ゆれの実態を把握するために、商品名のユニーク値の数を確認します。

In [383]:
print( len(uriage_data["item_name"].unique()) )

99


前のノック通り、99種類の商品名があることになっています。\
表記ゆれの種類を確認するために、ユニーク値を表示します。

In [384]:
uriage_data["item_name"].unique().sort()

item_name
str
""" 商品W"""
""" 商 品 n"""
""" 商品E"""
""" 商品M"""
""" 商品P"""
""" 商品S"""
""" 商品W"""
""" 商品X"""
"""商 品O"""


商品名の表記ゆれは次の2種類に大別されます。
* アルファベットが小文字になっている
* 様々な個所にスペースが入っている

この表記ゆれを修正します。

In [385]:
uriage_data = (
    uriage_data
    .with_columns(
        pl.col("item_name")
        .str.to_uppercase()# アルファベットの大文字に変換する
        .str.replace_all("　", "")# 全角スペースを削除する
        .str.replace_all(" ", "")# 半角スペースを削除する
        .alias("item_name")
    )
)

uriage_data.sort(by = ["item_name"])

purchase_date,item_name,item_price,customer_name,purchase_month
datetime[μs],str,i64,str,str
2019-06-13 18:02:34,"""商品A""",100,"""深井菜々美""","""201906"""
2019-05-11 19:42:07,"""商品A""",null,"""南部慶二""","""201905"""
2019-04-22 03:09:35,"""商品A""",null,"""平田鉄二""","""201904"""
2019-05-18 19:16:53,"""商品A""",null,"""深井照生""","""201905"""
2019-01-28 10:47:03,"""商品A""",100,"""大地礼子""","""201901"""
2019-06-11 12:57:24,"""商品A""",null,"""黄川田博之""","""201906"""
2019-06-29 10:01:52,"""商品A""",100,"""石田佑""","""201906"""
2019-01-12 03:26:36,"""商品A""",100,"""小平陽子""","""201901"""
2019-02-11 17:00:37,"""商品A""",100,"""原口俊二""","""201902"""


商品名の表記ゆれを修正できたことを確認するために、先ほどと同様にユニーク値とレコード数を表示します。

In [386]:
print( uriage_data["item_name"].unique() )
print( len(uriage_data["item_name"].unique()) )

shape: (26,)
Series: 'item_name' [str]
[
	"商品Q"
	"商品N"
	"商品U"
	"商品M"
	"商品H"
	"商品R"
	"商品V"
	"商品E"
	"商品C"
	"商品L"
	"商品G"
	"商品W"
	"商品A"
	"商品Y"
	"商品D"
	"商品F"
	"商品X"
	"商品P"
	"商品J"
	"商品Z"
	"商品I"
	"商品O"
	"商品K"
	"商品T"
	"商品B"
	"商品S"
]
26


商品名のユニーク値の数が26個なので、表記ゆれ修正が正しく完了したことを確認できました。

### ノック１５：金額欠損値の補完をしよう

商品名の修正が終わったので、次は商品価格の欠損値を修正しましょう。\
最初に、欠損値がどの程度あるのか確認します。

In [387]:
print(uriage_data.shape)
display(uriage_data.null_count())

(2999, 5)


purchase_date,item_name,item_price,customer_name,purchase_month
u32,u32,u32,u32,u32
0,0,387,0,0


2,999行中387行の欠損値が見つかりました。\
それなりの数の欠損値があります。

商品価格の欠損値を埋めるために、商品名と商品価格の辞書型を作成して、この辞書で欠損値を埋めます。

In [388]:
# 元のコードでは要素ごとに商品価格の最大値を検索していた
# 毎回検索するのは計算資源がもったいないので、あらかじめ商品名と商品価格の辞書を作成しておく
item_name_price = (
    uriage_data[["item_name", "item_price"]]
    .drop_nulls()
    .sort(by = "item_price", descending = True)
    .unique(subset = "item_name", keep = "first")
    .sort(by = "item_name", descending = False)
)

dict_name_price = {}
for name, price in zip(item_name_price["item_name"], item_name_price["item_price"]):
    dict_name_price[name] = price

# 商品価格の欠損値を埋める
uriage_data = (
    uriage_data
    .with_columns(
        pl.col("item_name")
        .replace(dict_name_price)
        .str.to_integer()# 文字列(商品名)の列を整形したので、数値型に変換する
        .alias("item_price")
    )
)    

uriage_data.head()

purchase_date,item_name,item_price,customer_name,purchase_month
datetime[μs],str,i64,str,str
2019-06-13 18:02:34,"""商品A""",100,"""深井菜々美""","""201906"""
2019-07-13 13:05:29,"""商品S""",1900,"""浅田賢二""","""201907"""
2019-05-11 19:42:07,"""商品A""",100,"""南部慶二""","""201905"""
2019-02-12 23:40:45,"""商品Z""",2600,"""麻生莉緒""","""201902"""
2019-04-22 03:09:35,"""商品A""",100,"""平田鉄二""","""201904"""


商品価格の欠損値埋めが上手くいったかどうかをかくにんするために、欠損値の数を表示します。

In [389]:
uriage_data.null_count()

purchase_date,item_name,item_price,customer_name,purchase_month
u32,u32,u32,u32,u32
0,0,0,0,0


各商品の商品価格を表示します。

In [390]:
for name in uriage_data["item_name"].unique().sort():
    print(name
          + "の最大値:" + str(uriage_data.filter(pl.col("item_name") == name)["item_price"].max())
          + "\tの最小値:" + str(uriage_data.filter(pl.col("item_name") == name)["item_price"].min()))

商品Aの最大値:100	の最小値:100
商品Bの最大値:200	の最小値:200
商品Cの最大値:300	の最小値:300
商品Dの最大値:400	の最小値:400
商品Eの最大値:500	の最小値:500
商品Fの最大値:600	の最小値:600
商品Gの最大値:700	の最小値:700
商品Hの最大値:800	の最小値:800
商品Iの最大値:900	の最小値:900
商品Jの最大値:1000	の最小値:1000
商品Kの最大値:1100	の最小値:1100
商品Lの最大値:1200	の最小値:1200
商品Mの最大値:1300	の最小値:1300
商品Nの最大値:1400	の最小値:1400
商品Oの最大値:1500	の最小値:1500
商品Pの最大値:1600	の最小値:1600
商品Qの最大値:1700	の最小値:1700
商品Rの最大値:1800	の最小値:1800
商品Sの最大値:1900	の最小値:1900
商品Tの最大値:2000	の最小値:2000
商品Uの最大値:2100	の最小値:2100
商品Vの最大値:2200	の最小値:2200
商品Wの最大値:2300	の最小値:2300
商品Xの最大値:2400	の最小値:2400
商品Yの最大値:2500	の最小値:2500
商品Zの最大値:2600	の最小値:2600


商品価格はアルファベットの順番 x 100のようです。

### ノック１６：顧客名の揺れを補正しよう

次は顧客名の修正です。\
まずは、顧客台帳のデータの様子を見てみます。

In [391]:
kokyaku_data["顧客名"].head()

顧客名
str
"""須賀ひとみ"""
"""岡田 敏也"""
"""芳賀 希"""
"""荻野 愛"""
"""栗田 憲一"""
"""梅沢 麻緒"""
"""相原 ひとり"""
"""新村 丈史"""
"""石川 まさみ"""


姓名の間の空白が自由なことが分かります。\
同様に、売上履歴の顧客名を確認してみます。

In [392]:
uriage_data["customer_name"].head()

customer_name
str
"""深井菜々美"""
"""浅田賢二"""
"""南部慶二"""
"""麻生莉緒"""
"""平田鉄二"""
"""堀江佑"""
"""深井照生"""
"""牧田玲那"""
"""堀北雅彦"""


売上履歴の顧客名はスペース無しのようです。\
データをそろえるために、顧客台帳の顧客名の空白を削除します。

In [393]:
kokyaku_data = (
    kokyaku_data
    .with_columns(
        pl.col("顧客名")
        .str.replace_all("　", "")
        .str.replace_all(" ", "")
        .alias("顧客名")
    )
)
kokyaku_data["顧客名"].head(n = 5)

顧客名
str
"""須賀ひとみ"""
"""岡田敏也"""
"""芳賀希"""
"""荻野愛"""
"""栗田憲一"""


顧客台帳の顧客名の修正が終わりましたので、売上履歴との横結合が上手くできるようになります。

### ノック１７：日付の揺れを補正しよう

このノックは書籍と異なる方法で日付の揺れを修正します。\
再度、顧客台帳の先頭行を表示します。

In [394]:
print(len(kokyaku_data))
display(kokyaku_data.head())

200


顧客名,かな,地域,メールアドレス,登録日
str,str,str,str,str
"""須賀ひとみ""","""すが ひとみ""","""H市""","""suga_hitomi@ex…","""2018/01/04"""
"""岡田敏也""","""おかだ としや""","""E市""","""okada_toshiya@…","""42782"""
"""芳賀希""","""はが のぞみ""","""A市""","""haga_nozomi@ex…","""2018/01/07"""
"""荻野愛""","""おぎの あい""","""F市""","""ogino_ai@examp…","""42872"""
"""栗田憲一""","""くりた けんいち""","""E市""","""kurita_kenichi…","""43127"""


Excel台帳への入力時にシリアル値で入力していた値が含まれている。\
このシリアル値も含めてDateTime型に変換する。

In [395]:
from datetime import datetime, timedelta

# Excelの日付形式をPythonのdatetime型に変換する関数を定義します。
def convert_excel_date_to_datetime(excel_date):
    return datetime(1899, 12, 30) + timedelta(days=int(excel_date))

# 日付と数字が混在したSeriesを、Excel書式の日付にに変換する
kokyaku_data = (
    kokyaku_data
    .with_columns(
        pl.col("登録日")
        #念の為文字列型にしておく
        .cast(pl.Utf8)
        # DateTime型に変換する
        .map_elements(
            lambda x:
            convert_excel_date_to_datetime(x)
            if x.isdigit()
            else datetime.strptime(x, "%Y/%m/%d")
        )
        .alias("登録日")
    )
)

kokyaku_data.head()

顧客名,かな,地域,メールアドレス,登録日
str,str,str,str,datetime[μs]
"""須賀ひとみ""","""すが ひとみ""","""H市""","""suga_hitomi@ex…",2018-01-04 00:00:00
"""岡田敏也""","""おかだ としや""","""E市""","""okada_toshiya@…",2017-02-16 00:00:00
"""芳賀希""","""はが のぞみ""","""A市""","""haga_nozomi@ex…",2018-01-07 00:00:00
"""荻野愛""","""おぎの あい""","""F市""","""ogino_ai@examp…",2017-05-17 00:00:00
"""栗田憲一""","""くりた けんいち""","""E市""","""kurita_kenichi…",2018-01-27 00:00:00


シリアル値も含めてDateTime型への変換が上手くいきました。\
分析のために登録年月の列を作成して、登録年月ごとの登録顧客数を集計します。

In [396]:
kokyaku_data = (
    kokyaku_data
    # 登録年月を加える
    .with_columns(
        pl.col("登録日")
        .dt.strftime("%Y%m")
        .alias("登録年月")
    )
)

rslt = (
    kokyaku_data
    # 登録年月ごとの新規登録顧客数を集計する
    .group_by(by = "登録年月")
    .agg( pl.col("顧客名").count() )
    .sort(by = "登録年月")
)
display(rslt)
print(len(kokyaku_data))

登録年月,顧客名
str,u32
"""201701""",15
"""201702""",11
"""201703""",14
"""201704""",15
"""201705""",14
"""201706""",13
"""201707""",17
"""201801""",13
"""201802""",15


200


これでシリアル値が混在していた日付の修正が終わりました。

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

顧客台帳と売上履歴を結合します。\
売上履歴の粒度が細かいので、こちらのデータを基準に横結合を行います。\

まず結合キーを確認するために、もう一度それぞれのデータの列名を表示します。

In [397]:
uriage_data.head(n = 1)

purchase_date,item_name,item_price,customer_name,purchase_month
datetime[μs],str,i64,str,str
2019-06-13 18:02:34,"""商品A""",100,"""深井菜々美""","""201906"""


In [398]:
kokyaku_data.head(n = 1)

顧客名,かな,地域,メールアドレス,登録日,登録年月
str,str,str,str,datetime[μs],str
"""須賀ひとみ""","""すが ひとみ""","""H市""","""suga_hitomi@ex…",2018-01-04 00:00:00,"""201801"""


売上履歴の"customer_name"と顧客台帳の"顧客名"が結合キーにできそうです。

データを結合します。

In [399]:
# 売上実績のcustomer_name列と、顧客名簿の顧客名列とをキーにして結合する
# 中間データを顧客名で保存するので、結合時にcustomer_name列の列名を変更する
join_data = (
    uriage_data
    .rename(mapping = {"customer_name": "顧客名"})
    .join(other = kokyaku_data, on="顧客名", how="left")
)

join_data.head()

purchase_date,item_name,item_price,顧客名,purchase_month,かな,地域,メールアドレス,登録日,登録年月
datetime[μs],str,i64,str,str,str,str,str,datetime[μs],str
2019-06-13 18:02:34,"""商品A""",100,"""深井菜々美""","""201906""","""ふかい ななみ""","""C市""","""fukai_nanami@e…",2017-01-26 00:00:00,"""201701"""
2019-07-13 13:05:29,"""商品S""",1900,"""浅田賢二""","""201907""","""あさだ けんじ""","""C市""","""asada_kenji@ex…",2018-04-07 00:00:00,"""201804"""
2019-05-11 19:42:07,"""商品A""",100,"""南部慶二""","""201905""","""なんぶ けいじ""","""A市""","""nannbu_keiji@e…",2018-06-19 00:00:00,"""201806"""
2019-02-12 23:40:45,"""商品Z""",2600,"""麻生莉緒""","""201902""","""あそう りお""","""D市""","""asou_rio@examp…",2018-07-22 00:00:00,"""201807"""
2019-04-22 03:09:35,"""商品A""",100,"""平田鉄二""","""201904""","""ひらた てつじ""","""D市""","""hirata_tetsuji…",2017-06-07 00:00:00,"""201706"""


### ノック１９：クレンジングしたデータをダンプしよう

ノック11から18まで複数の前処理を行いました。\
毎回全ての処理を行うのは非効率的なので、前処理結果を保存しましょう。

分析しやすくするために、列名を整理したデータフレームを作成します。

In [400]:
dump_data = join_data[["purchase_date", "purchase_month", "item_name", "item_price", "顧客名", "かな", "地域", "メールアドレス", "登録日"]]
dump_data.head()

purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
datetime[μs],str,str,i64,str,str,str,str,datetime[μs]
2019-06-13 18:02:34,"""201906""","""商品A""",100,"""深井菜々美""","""ふかい ななみ""","""C市""","""fukai_nanami@e…",2017-01-26 00:00:00
2019-07-13 13:05:29,"""201907""","""商品S""",1900,"""浅田賢二""","""あさだ けんじ""","""C市""","""asada_kenji@ex…",2018-04-07 00:00:00
2019-05-11 19:42:07,"""201905""","""商品A""",100,"""南部慶二""","""なんぶ けいじ""","""A市""","""nannbu_keiji@e…",2018-06-19 00:00:00
2019-02-12 23:40:45,"""201902""","""商品Z""",2600,"""麻生莉緒""","""あそう りお""","""D市""","""asou_rio@examp…",2018-07-22 00:00:00
2019-04-22 03:09:35,"""201904""","""商品A""",100,"""平田鉄二""","""ひらた てつじ""","""D市""","""hirata_tetsuji…",2017-06-07 00:00:00


データを保存しましょう。

In [401]:
dump_data.write_csv("dump_data.csv")

### ノック２０：データを集計しよう

前節までのノックで前処理を済ませたデータフレームを読み込んで、いくつかの集計を行いたいと思います。\
データを読み込みます。

In [402]:
import_data = pl.read_csv("dump_data.csv")
import_data.head()

purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
str,i64,str,i64,str,str,str,str,str
"""2019-06-13T18:…",201906,"""商品A""",100,"""深井菜々美""","""ふかい ななみ""","""C市""","""fukai_nanami@e…","""2017-01-26T00:…"
"""2019-07-13T13:…",201907,"""商品S""",1900,"""浅田賢二""","""あさだ けんじ""","""C市""","""asada_kenji@ex…","""2018-04-07T00:…"
"""2019-05-11T19:…",201905,"""商品A""",100,"""南部慶二""","""なんぶ けいじ""","""A市""","""nannbu_keiji@e…","""2018-06-19T00:…"
"""2019-02-12T23:…",201902,"""商品Z""",2600,"""麻生莉緒""","""あそう りお""","""D市""","""asou_rio@examp…","""2018-07-22T00:…"
"""2019-04-22T03:…",201904,"""商品A""",100,"""平田鉄二""","""ひらた てつじ""","""D市""","""hirata_tetsuji…","""2017-06-07T00:…"


In [403]:
byItem = (
    import_data
    .pivot(index = "purchase_month",
           columns = "item_name",
           values = "item_name",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byItem

purchase_month,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,商品K,商品L,商品M,商品N,商品O,商品P,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
i64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
201901,18,13,19,17,18,15,11,16,18,17,20,19,19,16,24,16,17,21,20,17,7,22,13,14,10,null
201902,19,14,26,21,16,14,14,17,12,14,16,11,15,20,19,19,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,14,20,22,13,11,21,23,16,20,12,23,18,16,21,16,null
201904,17,19,24,20,18,17,14,11,18,13,14,15,11,20,15,15,20,20,16,16,11,15,14,16,20,null
201905,24,14,16,14,19,18,23,15,16,11,18,13,18,19,18,20,13,22,18,16,16,9,21,16,20,null
201906,24,12,11,19,13,18,15,13,19,22,15,15,17,16,15,18,15,16,21,12,18,20,17,15,13,null
201907,20,20,17,17,12,17,19,19,19,23,12,17,11,15,22,26,15,19,23,21,13,28,16,18,12,null


In [404]:
byPrice = (
    import_data
    .pivot(index = "purchase_month",
           columns = "item_name",
           values = "item_price",
           aggregate_function = "sum",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byPrice

purchase_month,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,商品K,商品L,商品M,商品N,商品O,商品P,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,22000,22800,24700,22400,36000,25600,28900,37800,38000,34000,14700,48400,29900,33600,25000,null
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,17600,13200,19500,28000,28500,30400,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,15400,24000,28600,18200,16500,33600,39100,28800,38000,24000,48300,39600,36800,50400,40000,null
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,15400,18000,14300,28000,22500,24000,34000,36000,30400,32000,23100,33000,32200,38400,50000,null
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,19800,15600,23400,26600,27000,32000,22100,39600,34200,32000,33600,19800,48300,38400,50000,null
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,16500,18000,22100,22400,22500,28800,25500,28800,39900,24000,37800,44000,39100,36000,32500,null
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,13200,20400,14300,21000,33000,41600,25500,34200,43700,42000,27300,61600,36800,43200,30000,null


In [405]:
byCustomer = (
    import_data
    .pivot(index = "purchase_month",
           columns = "顧客名",
           values = "顧客名",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byCustomer

purchase_month,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,井川里穂,井本マサカズ,井村俊二,今茜,佐藤慶二,八木雅彦,内村まさみ,内田聡,南部慶二,原口俊二,古川信吾,合田光,吉岡サダヲ,吉村愛梨,和泉直人,唐沢景子,唐沢涼,土屋朝陽,城戸芳正,堀サンタマリア,堀内聡,堀北雅彦,堀江佑,外山広司,大倉晃司,大地礼子,大城ケンイチ,大山咲,大崎ヒカル,大滝麗奈,大西隆之介,奥光洋,宇野秀樹,小口豊,小川美菜,小平陽子,小松季衣,小松隼士,小栗正義,小町瞬,尾上勝久,尾形小雁,山口法子,山西花,岡慶太,岡村希,岡田敏也,岩井莉緒,岩佐孝太郎,岩城徹平,岩沢那奈,島孝太郎,島崎礼子,島本研二,島英嗣,島袋友以乃,川上りえ,川島友以乃,市田寿明,平田鉄二,平賀一哉,影山輝信,徳重優,志村サダヲ,戸塚美幸,手塚進,手塚雅之,新村丈史,新村美月,新谷智花,日比野徹,日野夏希,明石家明,星野美嘉,有馬徹平,望月真悠子,本多フミヤ,本橋直人,杉下悟志,杉田将也,村山知世,東光博,松元翔太,松居満,松岡ノブヒコ,松川綾女,松村聡,松沢育二,松田浩正,松谷愛子,板橋隆,林勇,栗田憲一,根岸仁晶,根岸莉央,根本博明,桑原桃子,梅村秀樹,梅沢麻緒,梅津淳子,森岡季衣,植木沙知絵,植村遥,楠哲平,楠高史,榊原しぼり,榎本薫,横田遥,水野メイサ,沖遥,河内さとみ,河村由樹,浅田賢二,浅見広司,浜田未華子,深井照生,深井菜々美,深沢ひろ子,深田信輔,清水佑,清水裕次郎,熊井憲史,熊倉明日,熊倉綾,片瀬長利,牧田玲那,田上美佐子,田崎菜々美,田畑正敏,田辺きみまろ,田辺光洋,白井俊二,白鳥りえ,相原ひとり,相川良介,矢沢恵梨香,矢部夏空,矢部惇,矢部美幸,石崎幸子,石川まさみ,石渡小雁,石田佑,石田花,石田郁恵,石野仁,磯野希,神原美嘉,福島友也,福本美幸,秋葉あき,稲田将也,立石茜,笹原しぼり,笹川照生,篠山雅功,米沢仁晶,綾瀬俊介,芦田博之,芳賀希,若杉徹,荻野愛,荻野愛菜,菅原誠治,藤広之,藤木一恵,西原未華子,西島知世,西脇礼子,谷本愛梨,赤木だん吉,赤木愛梨,進藤瞬,那須蒼甫,野本仁晶,金森なつみ,鈴木一哉,青山鉄洋,須賀ひとみ,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
i64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
201901,3,1,4,2,2,null,5,3,3,1,4,1,4,3,4,4,2,1,null,4,2,1,1,2,3,1,null,1,2,2,3,1,4,6,null,4,1,3,2,3,null,2,2,2,3,5,null,5,1,2,3,2,4,4,3,1,2,null,1,2,1,null,1,2,2,5,2,6,3,1,3,null,null,4,5,1,3,1,2,3,1,3,null,3,4,1,3,1,3,4,2,2,1,1,1,1,3,3,2,3,3,1,2,null,1,2,null,2,2,2,null,1,1,1,2,2,2,1,null,2,2,2,3,null,4,3,1,1,3,2,1,null,1,2,1,4,3,2,1,1,2,2,3,1,1,1,1,2,1,3,4,1,3,3,3,3,2,null,4,1,1,2,4,null,1,1,4,4,2,1,2,3,2,4,4,null,7,1,null,2,5,3,2,2,2,2,4,2,2,null,1,1,1,5,2,null,2,2,5
201902,9,1,2,2,1,4,2,1,null,4,2,3,1,1,3,2,3,2,4,2,1,2,null,1,5,1,1,3,1,3,3,1,3,1,2,2,1,1,1,1,1,3,null,2,1,3,6,1,4,3,2,5,2,2,5,5,4,4,4,1,null,null,null,3,3,1,5,2,3,4,2,2,3,1,5,1,5,1,null,1,5,2,null,null,1,1,2,4,null,null,2,4,5,4,1,4,3,2,3,2,1,4,3,2,1,null,3,1,4,1,3,1,1,2,1,1,3,null,4,3,5,null,2,3,3,3,3,1,null,2,7,4,1,1,3,5,3,2,4,2,null,4,null,2,3,2,null,1,2,3,5,5,3,2,2,5,3,4,2,2,5,2,1,null,1,2,2,3,2,1,1,2,2,7,1,1,2,1,1,1,3,3,2,1,2,4,1,4,3,4,null,3,2,null,1,2,4,null,1
201903,1,2,1,6,1,4,3,3,2,2,1,2,null,3,2,2,3,2,1,5,1,4,2,2,null,3,4,1,null,1,5,2,5,null,1,2,1,3,2,3,null,2,null,4,null,1,null,2,2,null,2,3,3,1,2,5,3,2,2,null,1,1,4,2,3,3,2,2,2,3,null,5,1,3,5,1,2,2,4,2,2,1,4,null,3,null,1,1,1,2,1,5,2,2,null,2,5,6,2,1,null,5,5,2,4,2,null,2,null,2,2,null,3,2,3,5,2,3,null,4,5,1,2,3,4,2,4,2,5,2,1,3,2,2,4,1,4,2,2,1,1,3,2,1,2,3,4,2,3,2,4,null,2,null,4,3,3,3,2,2,2,1,1,2,5,5,1,2,1,1,null,1,2,1,5,3,1,3,2,1,3,1,1,5,2,null,1,null,2,3,1,6,2,4,2,4,2,2,1
201904,null,3,1,2,null,2,2,null,3,2,2,1,4,1,3,1,1,4,5,3,1,2,1,3,4,4,4,6,null,null,1,4,3,1,2,null,2,2,2,1,2,4,1,1,2,2,3,2,2,1,3,4,3,null,1,null,3,3,1,1,1,2,null,3,5,5,3,3,1,3,1,1,2,2,4,2,1,2,1,2,3,1,3,null,1,3,1,1,3,4,1,2,2,1,1,2,4,null,5,3,2,4,1,4,1,1,4,4,null,5,null,1,5,null,6,1,null,3,1,1,5,2,4,4,3,null,1,2,1,4,3,1,2,1,2,5,1,1,2,null,4,2,1,null,4,null,3,1,4,2,null,null,3,1,1,null,1,1,1,2,2,null,2,1,5,2,4,2,1,2,2,4,2,3,1,1,1,4,null,1,1,3,4,3,null,1,3,2,3,2,4,2,3,4,3,2,1,2,null
201905,3,2,5,2,4,1,2,1,3,3,null,3,1,1,1,3,2,1,2,1,3,1,null,6,null,null,4,3,null,2,1,1,3,null,1,null,1,2,4,4,null,2,1,1,3,1,2,1,1,1,3,2,1,2,3,3,2,2,2,2,3,5,2,1,6,3,null,null,2,2,3,2,3,4,1,3,4,null,5,null,5,6,1,4,3,null,5,4,null,null,2,3,3,2,3,5,2,4,5,1,null,2,2,3,2,4,2,2,1,4,1,2,1,2,n

In [406]:
byRegion = (
    import_data
    .pivot(index = "purchase_month",
           columns = "地域",
           values = "地域",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byRegion

purchase_month,A市,B市,C市,D市,E市,F市,G市,H市
i64,u32,u32,u32,u32,u32,u32,u32,u32
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


最後に、この売上履歴の集計期間で購入していないユーザーがいないかチェックしてみます。\
この場合は、顧客台帳に売上履歴を横結合して、購入履歴が Null のレコードを抽出すれば確認できます。

In [409]:
away_data = (
    kokyaku_data
    .join(
        other = uriage_data.rename(mapping = {"customer_name": "顧客名"}),
        on = "顧客名",
        how = "left"
    )
)
away_data.filter( pl.col("purchase_date").is_null() )[["顧客名", "メールアドレス", "登録日"]]

顧客名,メールアドレス,登録日
str,str,datetime[μs]
"""福井美希""","""fukui_miki1@ex…",2019-04-23 00:00:00


1名の顧客の販売実績が無いことが分かりました。